# HeartAttack Prediction
### 속성 설명
1. Age : 나이
2. Sex : 성별
3. cp : 가슴 통증 종류
    - 1: 전형적인 협심증
    - 2: 불규칙적인 협심증
    - 3: 협심증이 아닌 통증
    - 4: 무증상
4. trtbps : 혈압
5. chol : 콜레스트롤 수치
6. fbs : 공복 혈당 (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
7. rest_ecg : resting electrocardiographic results (0, 1, 2)
8. exang : exercise induced angina (1 = yes; 0 = no)
9. output : 0= 심장마비 확률 적음, 1= 심장마비 확률 높음
 
(angina/협심증 : 심장의 혈액 순환로가 되는 관상동맥이 좁아져 갑작스럽게 흉부 통증 또는 압박감을 느끼는 상태)

- 역할 분담 (예정)
    - 이예현 : 데이터 시각화, 주석 작성
    - 정해성 : 모델 학습, 예측, 평가

- 출처 : https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset

### 데이터 시각화
---


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### 머신러닝 모델 학습
---

In [2]:
import sklearn